In [86]:
import os
import numpy as np
import sys
import socket
import pickle
import math
import os
import json
from tqdm import tqdm
import torch
import k3d
from shutil import copytree

from pytorch3d.io import load_obj,save_ply
from pytorch3d.ops import sample_points_from_meshes,knn_points,sample_farthest_points
from pytorch3d.structures import Pointclouds,Meshes
import trimesh

from leveraging_geometry_for_shape_estimation.models.represent_3d_object.extract_points_from_models import find_edge_points,load_points_normals,sample_points_from_lines,repair_mesh,plot_points,different_normals_around_points,vis_representative_normals
from leveraging_geometry_for_shape_estimation.models.represent_3d_object.moller_trumbore import inside_mesh
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [87]:
with open("/scratch2/fml35/experiments/leveraging_geometry_for_shape/exp_117_scannet_models/models/model_list.json",'r') as f:
    model_list = json.load(f)["models"]

In [88]:
n_points = 50000
n_points_ref = 50000
n_nn = 5
angle_threshold_degree = 20
scannet_path = '/scratch2/fml35/datasets/shapenet_v2/ShapeNetRenamed/'
j = 0
device = torch.device("cpu")
print(model_list[j]["model"])

model_list[j]["model"] = "model_fixed/cabinet/4ec97e83a1e9725c77a332e31ab74b68/model_normalized.obj"


model/bathtub/38773e1ff9871cb33321831d2245cf06/model_normalized.obj


In [89]:

sample_points,sample_normals = load_points_normals(scannet_path + model_list[j]["model"],n_points)
edge_points = find_edge_points(sample_points,sample_normals,n_nn,angle_threshold_degree,device)

torch.Size([89906, 3])
faces torch.Size([179808, 3])
vertices (88982, 3)
faces (177960, 3)


In [90]:
print(sample_points.shape)
print(sample_normals.shape)
print(edge_points.shape)

torch.Size([50000, 3])
torch.Size([50000, 3])
torch.Size([6439, 3])


In [91]:
verts_torch,faces_torch,_ = load_obj(scannet_path + model_list[j]["model"],load_textures=False)
# faces_fixed = repair_mesh(verts_torch,faces_torch[0])


In [92]:
plot_points(edge_points)
furthest_points,indices = sample_farthest_points(edge_points.unsqueeze(0),K=torch.Tensor([300]))
print(furthest_points.shape)
plot_points(furthest_points.squeeze(0))

Output()

torch.Size([1, 300, 3])


Output()

In [96]:
ref_points,ref_normals = load_points_normals(scannet_path + model_list[j]["model"],n_points_ref)



In [97]:

representative_normals = different_normals_around_points(furthest_points.squeeze(0),ref_points,ref_normals,n_nn=20,n_normals_per_point=3)

print(representative_normals.shape)
# print(representative_normals)[:10]

torch.Size([300, 3, 3])


In [98]:


verts_torch,faces_torch,_ = load_obj(scannet_path + model_list[j]["model"],load_textures=False)
# faces = faces_torch[0]
# vertices,faces = repair_mesh(verts_torch,faces_torch[0])
# plt_mesh = k3d.mesh(vertices.numpy(),faces.numpy())
plt_mesh = k3d.mesh(verts_torch.numpy(),faces_torch[0].numpy())
vis_representative_normals(furthest_points.squeeze(0),representative_normals,plt_mesh)  

/scratch/fml35/environments/shape_env_octo/lib/python3.6/site-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "int64" does not match required type "uint32". A coerced copy has been created.
  np.dtype(self.dtype).name))


Output()

In [ ]:
# def ensure_normals_facing_out(points,normals,vertices,faces,mesh=None):

#     plot = k3d.plot()

#     for i in range(1):
#         # print(points.shape)
#         # print(normals.shape)
#         lines = torch.cat([points,points+normals[:,i]*0.05],dim=1)
#         # print(lines.shape)
#         check_points = sample_points_from_lines(lines,points_per_line=30)
#         check_points = check_points.view(-1,3)
#         check_points = check_points[10:11]
#         # print('checl points',check_points.shape)
#         inside,intersection = inside_mesh(check_points,vertices,faces)
#         print(inside.shape)
#         # print('inside',inside[:100])
#         colors = inside * 255

#         indices = torch.nonzero(intersection.squeeze())
#         indices = indices.squeeze()
#         # print('indices',indices)
#         # print('faces',faces.shape)
#         # print('faces[indices]',faces[indices])
#         # print(faces[indices])
#         points_faces_hit = vertices[faces[indices].view(-1)]
#         # colors = get_colors(values,default_range=default_range)
#         print('faces[indices]',faces[indices])
#         print('points_faces_hit.shape',points_faces_hit.shape)
#         print('points_faces_hit',points_faces_hit)
#         # colors = np.round(np.array(colors)*255).astype(int)[:,:3]
#         # multi = np.array([1,256,256*256])
#         # colors = np.dot(colors,multi)
#         plot += k3d.points(points_faces_hit, point_size=0.03,color=0xff0000, shader="flat")

#         plot += k3d.points(check_points, point_size=0.003,colors=colors, shader="flat")
#         if mesh is not None:
#             plot += mesh

#     plot.display()

# # print(faces_torch[0])
# ensure_normals_facing_out(furthest_points.squeeze(0)[3:4],representative_normals[3:4],verts_torch,faces_torch[0],plt_mesh) 

In [ ]:
points_normals = np.load('/scratch2/fml35/experiments/leveraging_geometry_for_shape/exp_117_scannet_models/models/representation_points_and_normals/exp_01/points_and_normals/{}.npz'.format(model_list[j]["name"]))

In [ ]:
print(model_list[j]["model"])
print(points_normals["points"].shape)
print(points_normals["normals"].shape)
print(points_normals["normals"][:10])

model/cabinet/4ec97e83a1e9725c77a332e31ab74b68/model_normalized.obj
(300, 3)
(300, 3, 3)
[[[ 4.6247542e-03 -9.7900903e-01 -2.0376441e-01]
  [-4.0971637e-03  9.6985346e-01  2.4365473e-01]
  [-7.9515949e-03  1.7119370e-01  9.8520529e-01]]

 [[-5.7094508e-01  4.3822411e-01  6.9424874e-01]
  [-1.2260204e-03 -9.9977660e-01  2.1101112e-02]
  [ 0.0000000e+00 -1.5198085e-01  9.8838341e-01]]

 [[ 5.5322784e-01  4.5818186e-01  6.9570702e-01]
  [ 3.6097512e-02 -9.9842054e-01  4.3049473e-02]
  [ 6.0144728e-01 -6.4008790e-01  4.7806776e-01]]

 [[-7.0243472e-01  7.1174824e-01  0.0000000e+00]
  [-3.0554088e-02 -9.9953312e-01  8.3858453e-05]
  [-9.9070174e-01 -1.3605183e-01 -1.2557355e-10]]

 [[ 7.0227164e-01  7.1190906e-01  0.0000000e+00]
  [ 2.5627907e-02 -9.9967158e-01 -3.1588966e-06]
  [ 7.6705152e-01 -6.4158541e-01  0.0000000e+00]]

 [[-6.3602300e-03 -8.6518967e-01  5.0140446e-01]
  [-2.3793674e-03  9.6195030e-01 -2.7321407e-01]
  [ 9.5944321e-03  4.5346594e-01 -8.9122188e-01]]

 [[ 3.0866206e-01

In [ ]:
verts_torch,faces_torch,_ = load_obj(scannet_path + model_list[j]["model"],load_textures=False)
# faces = faces_torch[0]
vertices,faces = repair_mesh(verts_torch,faces_torch[0])
plt_mesh = k3d.mesh(vertices.numpy(),faces.numpy())
vis_representative_normals(torch.from_numpy(points_normals['points']),torch.from_numpy(points_normals['normals']),plt_mesh)

broken [5651 5652 5469 5470 5649 5650 5512 5513]
broken [5651 5652 5469 5470 5649 5650 5512 5513]


Output()